## 包含测距噪声的灵敏度曲线
这个灵敏度曲线用于探测器的优化，主要参考空间中心张佳峰师兄的优化文章，文章链接：https://www.mdpi.com/1424-8220/23/23/9431。
顶层灵敏度曲线只包含三个参数，探测器臂长$L$,加速度噪声$S_a$,测距噪声$S_x$。
这篇文章实际上只包含了测距噪声，如果可以最好加上加速度噪声。
@Copyright: ICTP-ap UCAS Junxi SHI

In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [2]:
pi = np.pi
GM_sun = 1.3271244*1e20 
c =2.9979246*1e8
M_sun =1.9884099*1e30
G = 6.6743*1e-11
pc= 3.0856776*1e16
e = 1.6e-19   #库伦

In [3]:
#这里首先给出Robson文章中的灵敏度曲线，随后将把Sx的公式整合在里面
def PSD_Lisa(f):
    
    """
    From https://arxiv.org/pdf/1803.01944.pdf. 
    """

    L = 2.5*10**9   # Length of LISA arm
    f0 = c/(2*pi*L)   
    Sx = 1.5*10**-11
    Sa = 3*10**-15
    Poms = (Sx**2)*(1 + ((2*10**-3)/f)**4)  # Optical Metrology Sensor
    Pacc = Sa**2*(1 + (4*10**-3/(10*f))**2)*(1 + (f/(8*10**-3))**4)  # Acceleration Noise
    Sc = 9*10**(-45)*f**(-7/3)*np.exp(-f**0.171 + 292*f*np.sin(1020*f)) * (1 \
                                            + np.tanh(1680*(0.00215 - f)))   # Confusion noise
    alpha = 0.171
    beta = 292
    k =1020
    gamma = 1680
    f_k = 0.00215 
    PSD = ((10/(3*L*L))*(Poms + (4*Pacc)/(np.power(2*pi*f,4)))*(1 + 0.6*(f/f0)*(f/f0)) + Sc) # PSD
        
    # Handling the zeroth frequency bin
    
    where_are_NaNs = np.isnan(PSD) 
    PSD[where_are_NaNs] = 1e100    # at f = 0, PSD is infinite. Approximate infinity and burn my 
                                   # mathematics degree to the ground. 
    
    return PSD



### 首先这里把所需的噪声模型打一遍，估计写文章的时候也是需要打的
这里首先涉及的是次顶层的，直接与
$$

In [17]:
#这里给定太极的一些固定参数，如果可以增加LISA的更好
eps_het = 0.8   #Heterodyne interference efficiency
R_pd = 0.68     #A/W    Photodiode responsivity
RIN = 1e-8      #Relative intensity laser noise
N_pd = 4.0e-12      #Photo detector phase number
U_pd = 2.0e-9   #V/sqrt(Hz)  Photodetectorvoltagenoise
C_pd = 10.0e-12 #F    Photodiode capacitance
I_pd = 1.5e-12  #A/sqrt(Hz)     Currentnoise
P_tel = 2.0     #W   Laser power transmitted through the telescope
eps_opt = 0.853 #Totalopticalefficiency
L = 3e9         #m
D = 0.4         #m     Diameter of telescop

In [18]:
from scipy.special import jv
m = 0   #modulation depth

lamb = 1064e-9    #激光波长
   
#f_uso = 0   #ultrastable oscillator frequency
def Sa(paras):
    f_low,f_up,f_adc,f_pt,f_uso,m,P_loc = paras     #一共七个优化参数
    f_het = f_up    #beat-note frequency
    j0 = jv(0,m)    #贝塞尔函数,下同
    j1 = jv(1,m)
   
    P_rec = 0.4073*pi**2*D**4*P_tel*eps_opt/(8*L**2*lamb**2)     #吸收功率

    #相位噪声
    phi_sn = np.sqrt(2*e*(P_loc+P_rec)/(R_pd*eps_het*P_loc*P_rec))
    phi_rin = RIN*np.sqrt((P_loc**2+P_rec**2)/(2*eps_het*P_loc*P_rec))
    phi_en = (np.sqrt(2*N_pd)/R_pd)*np.sqrt((I_pd**2+2*pi*C_pd*f_up*U_pd)/(eps_het*P_loc*P_rec))
    phi_tot = np.sqrt(phi_sn**2+phi_rin**2+phi_en**2)

    #carrier noise and sideband noise
    Sa_car = (lamb/2/pi)*(1/j0**2)*phi_tot
    Sa_sb = (1/np.sqrt(2))*(lamb/2/pi)*(f_het/f_uso)*(1/j1**2)*phi_tot
    
    return Sa_car + Sa_sb

para_test = [3.0e6,25.0e6,92.0e6,90.0e6,4.14e9,0.44,2.06e-3]
print(Sa(para_test))


3.779756903224823e-12


In [16]:
import numpy as np
from scipy.special import jv
from math import pi


e_charge = 1.6e-19      # 库仑 (C)
eps_het = 0.8           # 无量纲
R_pd = 0.68             # A/W
RIN = 1e-8              # W/Hz^0.5
N_pd = 4e-12            # rad^2/Hz
U_pd = 2e-9             # V/Hz^0.5
C_pd = 10e-12           # F
Ipd = 1.5e-12           # A/Hz^0.5
P_tel = 2.0             # W
eps_opt = 0.853         # 无量纲
L = 3e9                 # m
D = 0.4                 # m (40 cm 转换为米)
lamb = 1064e-9          # m
delta_f = 1.0           # Hz

def Sa(paras):
    f_low, f_up, f_adc, f_pt, f_uso, m, P_loc = paras
    f_het = f_up  # Beat-note frequency

    # 贝塞尔函数
    j0 = jv(0, m)
    j1 = jv(1, m)


  
    P_rec = 0.4073 * pi**2 * D**4 * P_tel * eps_opt / (8 * L**2 * lamb**2)
  

    phi_sn = np.sqrt(2 * e_charge * (P_loc + P_rec) / (R_pd * eps_het * P_loc * P_rec))
    phi_rin = RIN * np.sqrt((P_loc**2 + P_rec**2) / (2 * eps_het * P_loc * P_rec))
    phi_en = (np.sqrt(2 * N_pd * delta_f) / R_pd) * np.sqrt((Ipd**2 + 2 * pi * C_pd * f_up * U_pd) / (eps_het * P_loc * P_rec))

   

    # 总相位噪声
    phi_tot = np.sqrt(phi_sn**2 + phi_rin**2 + phi_en**2)
    print(f"总相位噪声 phi_tot: {phi_tot:.3e} rad")

    # 位移噪声计算
    Sa_car = (lamb / (2 * pi)) * (1 / j0**2) * phi_tot
    Sa_sb = (1 / np.sqrt(2)) * (lamb / (2 * pi)) * (f_het / f_uso) * (1 / j1**2) * phi_tot

    print(f"位移噪声: Sa_car={Sa_car:.3e} m, Sa_sb={Sa_sb:.3e} m")

    return Sa_car + Sa_sb

# 测试参数
para_test = [3.0e6, 25.0e6, 92.0e6, 90.0e6, 4.14e9, 0.44, 2.06e-3]  # [f_low, f_up, f_adc, f_pt, f_uso, m, P_loc]
sa_result = Sa(para_test)
print(f"总 Sa: {sa_result:.3e} meters")


总相位噪声 phi_tot: 1.866e-05 rad
位移噪声: Sa_car=3.485e-12 m, Sa_sb=2.927e-13 m
总 Sa: 3.778e-12 meters
